<a href="https://colab.research.google.com/github/saurabhsingh1411/yolo_v3_objecct_detection/blob/main/object_detection_opencv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importing intial lib 

In [2]:
import tensorflow as tf
import numpy as np
from PIL import Image,ImageDraw,ImageFont

from IPython.display import display
from seaborn import color_palette

import cv2

Model hyper parameters 

In [3]:
_BATCH_NORM_DECAY=0.9
_BATCH_NORM_EPSILON=1e-05
_LEAKY_RELU=0.1
_ANCHORS=[(10, 13), (16, 30), (33, 23),
            (30, 61), (62, 45), (59, 119),
            (116, 90), (156, 198), (373, 326)]

_MODEL_SIZE=(416,416)



## Batch norm and fixed padding

In [5]:
def batch_norm(inputs,training,data_format):


  '''performs batch normalization using as standrad set of parameters'''

  return tf.layers.batch_normalization(
        inputs=inputs, axis=1 if data_format == 'channels_first' else 3,
        momentum=_BATCH_NORM_DECAY, epsilon=_BATCH_NORM_EPSILON,
        scale=True, training=training)

In [7]:
def fixed_padding(inputs,kernel_size,data_format):

  pad_total=kernel_size-1
  pad_beg=pad_total//2
  pad_end=pad_total-pad_beg


  if data_format=='channels_first':

    padded_inputs=tf.pad(inputs,[[0, 0], [0, 0],
                                        [pad_beg, pad_end],
                                        [pad_beg, pad_end]])
    
  else :
    padded_inputs=tf.pad(inputs,[[0, 0], [pad_beg, pad_end],
                                        [pad_beg, pad_end], [0, 0]])
    
  return padded_inputs

In [8]:
def conv2d_fixed_padding(inputs, filters, kernel_size, data_format, strides=1):
    """Strided 2-D convolution with explicit padding."""
    if strides > 1:
        inputs = fixed_padding(inputs, kernel_size, data_format)

    return tf.layers.conv2d(
        inputs=inputs, filters=filters, kernel_size=kernel_size,
        strides=strides, padding=('SAME' if strides == 1 else 'VALID'),
        use_bias=False, data_format=data_format)